In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Célula de Configuração do Monitoramento (adicionar no início)

# 1. Instalar bibliotecas para monitoramento
!pip install psutil pynvml

# 2. Importar tudo o que vamos precisar
import time
import threading
import psutil
import pynvml
import pandas as pd
import matplotlib.pyplot as plt

# 3. Definir a classe que fará o monitoramento
class ResourceMonitor:
    def __init__(self, interval=5):
        self.interval = interval
        self.data = []
        self._stop_event = threading.Event()
        self.thread = threading.Thread(target=self.run, daemon=True)

        # Inicializa a NVML para monitoramento da GPU
        try:
            pynvml.nvmlInit()
            self.gpu_count = pynvml.nvmlDeviceGetCount()
        except pynvml.NVMLError:
            self.gpu_count = 0
            print("AVISO: Placa NVIDIA não encontrada ou driver indisponível. O monitoramento da GPU será desativado.")

    def _get_gpu_ram_usage(self):
        if self.gpu_count == 0:
            return 0
        handle = pynvml.nvmlDeviceGetHandleByIndex(0)
        info = pynvml.nvmlDeviceGetMemoryInfo(handle)
        return info.used / (1024**3)  # Convertido para GB

    def run(self):
        """O método que roda em segundo plano para coletar dados."""
        start_time = time.time()
        while not self._stop_event.is_set():
            timestamp = time.time() - start_time

            # Coleta de dados
            sys_ram_used = psutil.virtual_memory().used / (1024**3) # GB
            gpu_ram_used = self._get_gpu_ram_usage() # GB
            disk_used = psutil.disk_usage('/').used / (1024**3) # GB

            self.data.append([timestamp, sys_ram_used, gpu_ram_used, disk_used])
            time.sleep(self.interval)

        if self.gpu_count > 0:
            pynvml.nvmlShutdown()

    def start(self):
        """Inicia o monitoramento."""
        print("Iniciando monitoramento de recursos...")
        self.thread.start()

    def stop(self):
        """Para o monitoramento."""
        self._stop_event.set()
        self.thread.join()
        print("Monitoramento de recursos finalizado.")
        return pd.DataFrame(self.data, columns=['Tempo (s)', 'RAM Sistema (GB)', 'RAM GPU (GB)', 'Disco (GB)'])

    def plot(self):
        """Plota os dados coletados."""
        df = self.stop()

        if df.empty:
            print("Nenhum dado de monitoramento foi coletado.")
            return

        fig, axes = plt.subplots(3, 1, figsize=(12, 15), sharex=True)
        fig.suptitle('Utilização de Recursos do Sistema Durante a Execução', fontsize=16)

        # Gráfico de RAM do Sistema
        axes[0].plot(df['Tempo (s)'], df['RAM Sistema (GB)'], label='RAM do Sistema Utilizada', color='blue')
        axes[0].set_ylabel('Uso (GB)')
        axes[0].set_title('Uso de RAM do Sistema')
        axes[0].grid(True)
        axes[0].legend()
        axes[0].fill_between(df['Tempo (s)'], df['RAM Sistema (GB)'], alpha=0.1, color='blue')

        # Gráfico de RAM da GPU
        if self.gpu_count > 0:
            axes[1].plot(df['Tempo (s)'], df['RAM GPU (GB)'], label='RAM da GPU Utilizada', color='green')
        else:
            axes[1].text(0.5, 0.5, 'Monitoramento de GPU não disponível', ha='center', va='center')
        axes[1].set_ylabel('Uso (GB)')
        axes[1].set_title('Uso de RAM da GPU')
        axes[1].grid(True)
        axes[1].legend()
        axes[1].fill_between(df['Tempo (s)'], df['RAM GPU (GB)'], alpha=0.1, color='green')

        # Gráfico de Uso de Disco
        axes[2].plot(df['Tempo (s)'], df['Disco (GB)'], label='Espaço em Disco Utilizado', color='red')
        axes[2].set_xlabel('Tempo (segundos)')
        axes[2].set_ylabel('Uso (GB)')
        axes[2].set_title('Uso de Disco')
        axes[2].grid(True)
        axes[2].legend()
        axes[2].fill_between(df['Tempo (s)'], df['Disco (GB)'], alpha=0.1, color='red')

        plt.tight_layout(rect=[0, 0, 1, 0.96])
        plt.show()

# 4. Iniciar o cronômetro e o monitoramento
# (Coloque estas 2 linhas logo antes do seu código principal começar a rodar)
tempo_inicial = time.time()
monitor = ResourceMonitor(interval=5) # O 'interval' é em segundos
monitor.start()

In [ ]:

!pip install midi2audio

# Imports
import os
import numpy as np
import pandas as pd
from collections import Counter
import random
import warnings
from music21 import converter, instrument, note, chord, stream
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.utils
from midi2audio import FluidSynth
from google.colab import drive
from tqdm import tqdm # Importação da tqdm

# Configurações
warnings.filterwarnings("ignore")
np.random.seed(42)

# Caminhos
dataset_path = "/content/drive/MyDrive/maestro-full" # Caminho para a pasta do MAESTRO
soundfont_path = "/content/drive/MyDrive/soundfonts"
output_midi = "Melody_Generated.mid"
output_wav = "Melody_Generated.wav"

In [ ]:
import multiprocessing
from functools import partial

# --- CÉLULA DE CARREGAMENTO OTIMIZADA ---

# Função auxiliar que processa um único arquivo.
# Ela precisa ser definida fora do loop principal para funcionar com multiprocessing.
def parse_midi_file(file_path, show_warnings=False):
    try:
        midi = converter.parse(file_path)
        return midi
    except Exception as e:
        if show_warnings:
            # Pega apenas o nome do arquivo para a mensagem de erro
            file_name = os.path.basename(file_path)
            print(f"\nAVISO: Não foi possível processar o arquivo {file_name}. Erro: {e}")
        return None

# Iniciar o cronômetro e o monitoramento
tempo_inicial = time.time()
monitor = ResourceMonitor(interval=5)
monitor.start()

# Carregar MIDIs com barra de progresso e tratamento de erros
print(f"Carregando arquivos MIDI de: {dataset_path}")

try:
    file_list = [os.path.join(dataset_path, i) for i in os.listdir(dataset_path) if i.lower().endswith(('.mid', '.midi'))]

    # Define o número de processos (geralmente o número de núcleos da CPU é uma boa escolha)
    # Deixe None para usar todos os núcleos disponíveis
    num_processes = multiprocessing.cpu_count()
    print(f"Utilizando {num_processes} núcleos para acelerar o carregamento...")

    # Cria um pool de processos para executar a tarefa em paralelo
    # O 'if __name__ == "__main__":' é importante no Jupyter em alguns sistemas
    if __name__ == "__main__":
        with multiprocessing.Pool(processes=num_processes) as pool:
            # Usa pool.imap para processar a lista e manter a barra de progresso (tqdm)
            # O resultado será uma lista de objetos midi (ou None para arquivos com erro)
            results = list(tqdm(pool.imap(parse_midi_file, file_list), total=len(file_list), desc="Processando arquivos"))

    # Filtra os resultados para remover os arquivos que falharam (valor None)
    all_midis = [midi for midi in results if midi is not None]

    print(f"\nCarregamento concluído. {len(all_midis)} arquivos foram carregados com sucesso.")

except FileNotFoundError:
    print(f"ERRO: O diretório '{dataset_path}' não foi encontrado. Verifique o caminho na Célula 3.")

In [ ]:
# Extrair notas
def extract_notes(file):
    notes = []
    for j in file:
        songs = instrument.partitionByInstrument(j)
        for part in songs.parts:
            pick = part.recurse()
            for element in pick:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append(".".join(str(n) for n in element.normalOrder))
    return notes

Corpus = extract_notes(all_midis)
print("Total notes in all the Maestro midis:", len(Corpus))

# Remover notas raras
count_num = Counter(Corpus)
rare_note = [k for k, v in count_num.items() if v < 100]
Corpus = [element for element in Corpus if element not in rare_note]

# Mapeamento
symb = sorted(list(set(Corpus)))
mapping = dict((c, i) for i, c in enumerate(symb))
reverse_mapping = dict((i, c) for i, c in enumerate(symb))

In [ ]:
# Preparar dados
length = 40
features = []
targets = []

for i in range(0, len(Corpus) - length, 1):
    feature = Corpus[i:i + length]
    target = Corpus[i + length]
    features.append([mapping[j] for j in feature])
    targets.append(mapping[target])

X = np.reshape(features, (len(targets), length, 1)) / float(len(symb))
y = tensorflow.keras.utils.to_categorical(targets)

X_train, X_seed, y_train, y_seed = train_test_split(X, y, test_size=0.2, random_state=42)

# CÓDIGO NOVO (MAIS PROFUNDO E MAIS LARGO)
model = Sequential()
# Camada 1 (Larga)
model.add(LSTM(1024, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
# Camada 2 (Intermediária)
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.2))
# Camada 3 (Final Recorrente)
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.001) # Defina o Adam com a taxa de aprendizado que planejamos
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [ ]:
# LINHA ALTERADA
history = model.fit(X_train, y_train, batch_size=256, epochs=100) # Alterado de 50 para 100

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# --- Dados extraídos do seu log de treinamento ---
# (LSTM + MAESTRO, Rodada 2 - 100 épocas)
epochs = list(range(1, 101))
loss_values = [
    5.4411, 5.4157, 5.2680, 5.2018, 5.1140, 5.0411, 4.9991, 4.9693, 4.9417, 4.9156,
    4.8932, 4.8751, 4.8576, 4.8452, 4.8327, 4.8223, 4.8111, 4.7999, 4.7922, 4.7839,
    4.7762, 4.7681, 4.7596, 4.7535, 4.7461, 4.7420, 4.7336, 4.7281, 4.7226, 4.7176,
    4.7096, 4.7047, 4.6994, 4.6941, 4.6892, 4.6830, 4.6787, 4.6745, 4.6704, 4.6658,
    4.6606, 4.6555, 4.6522, 4.6477, 4.6436, 4.6391, 4.6358, 4.6302, 4.6267, 4.6241,
    4.6208, 4.6188, 4.6135, 4.6118, 4.6072, 4.6040, 4.5990, 4.5976, 4.5954, 4.5912,
    4.5890, 4.5875, 4.5846, 4.5803, 4.5781, 4.5757, 4.5748, 4.5698, 4.5699, 4.5691,
    4.5634, 4.5624, 4.5600, 4.5586, 4.5558, 4.5555, 4.5531, 4.5514, 4.5463, 4.5486,
    4.5455, 4.5440, 4.5444, 4.5396, 4.5375, 4.5347, 4.5362, 4.5353, 4.5341, 4.5333,
    4.5270, 4.5278, 4.5238, 4.5282, 4.5266, 4.5221, 4.5204, 4.5165, 4.5158, 4.5162
]
accuracy_values = [
    0.0131, 0.0134, 0.0164, 0.0210, 0.0275, 0.0336, 0.0369, 0.0397, 0.0417, 0.0437,
    0.0456, 0.0475, 0.0488, 0.0501, 0.0509, 0.0524, 0.0533, 0.0541, 0.0549, 0.0558,
    0.0567, 0.0572, 0.0584, 0.0587, 0.0596, 0.0600, 0.0604, 0.0610, 0.0618, 0.0624,
    0.0633, 0.0639, 0.0641, 0.0645, 0.0652, 0.0659, 0.0660, 0.0669, 0.0669, 0.0675,
    0.0678, 0.0685, 0.0687, 0.0691, 0.0693, 0.0700, 0.0707, 0.0704, 0.0711, 0.0711,
    0.0719, 0.0721, 0.0724, 0.0726, 0.0732, 0.0733, 0.0741, 0.0741, 0.0742, 0.0749,
    0.0749, 0.0753, 0.0755, 0.0762, 0.0758, 0.0764, 0.0766, 0.0768, 0.0772, 0.0770,
    0.0776, 0.0778, 0.0779, 0.0784, 0.0781, 0.0785, 0.0786, 0.0788, 0.0794, 0.0793,
    0.0794, 0.0797, 0.0796, 0.0802, 0.0803, 0.0806, 0.0807, 0.0811, 0.0808, 0.0810,
    0.0815, 0.0815, 0.0815, 0.0813, 0.0815, 0.0821, 0.0819, 0.0823, 0.0826, 0.0825
]
# ----------------------------------------------------

# --- Criar a Figura com dois subplots (um em cima do outro) ---
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

# --- Gráfico 1: Perda (Loss) ---
ax1.plot(epochs, loss_values, 'r.-', label='Perda de Treinamento')
ax1.set_title('Curva de Perda (Loss) por Época', fontsize=16)
ax1.set_ylabel('Valor da Perda (Loss)', fontsize=12)
ax1.grid(True, linestyle='--', alpha=0.6)
ax1.legend()
# Definir marcadores do eixo X para ambos os gráficos
ax1.set_xticks(np.arange(0, 101, 10)) # Marcadores a cada 10 épocas

# --- Gráfico 2: Acurácia (Accuracy) ---
ax2.plot(epochs, accuracy_values, 'g.-', label='Acurácia de Treinamento')
ax2.set_title('Curva de Acurácia por Epoch', fontsize=16)
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Acurácia', fontsize=12)
ax2.grid(True, linestyle='--', alpha=0.6)
ax2.legend()

# Ajustar o layout e exibir
plt.tight_layout()
plt.show()

In [ ]:
# Gerar música
def chords_n_notes(Snippet):
    Melody = []
    offset = 0
    for i in Snippet:
        if ("." in i or i.isdigit()):
            chord_notes = i.split(".")
            notes = [note.Note(int(j)) for j in chord_notes]
            chord_snip = chord.Chord(notes)
            chord_snip.offset = offset
            Melody.append(chord_snip)
        else:
            note_snip = note.Note(i)
            note_snip.offset = offset
            Melody.append(note_snip)
        offset += 1
    return stream.Stream(Melody)

def Malody_Generator(Note_Count):
    seed = X_seed[np.random.randint(0, len(X_seed)-1)]
    Notes_Generated = []
    for _ in range(Note_Count):
        seed = seed.reshape(1, length, 1)
        prediction = model.predict(seed, verbose=0)[0]
        prediction = np.log(prediction + 1e-8)
        exp_preds = np.exp(prediction)
        prediction = exp_preds / np.sum(exp_preds)
        index = np.argmax(prediction)
        Notes_Generated.append(index)
        seed = np.insert(seed[0], len(seed[0]), index / float(len(symb)))
        seed = seed[1:]
    Music = [reverse_mapping[char] for char in Notes_Generated]
    Melody = chords_n_notes(Music)
    return Music, Melody

Music_notes, Melody = Malody_Generator(100)
Melody.write('midi', output_midi)


In [ ]:
!apt-get update
!apt-get install fluidsynth

In [ ]:
# Converter .mid para .wav
fs = FluidSynth(soundfont_path)
fs.midi_to_audio(output_midi, output_wav)
print(f"Arquivo .wav gerado: {output_wav}")

# Reproduzir no Colab
import IPython.display as ipd
ipd.Audio(output_wav)


In [ ]:
# Célula de Finalização e Plotagem (adicionar no final)

# 1. Parar o cronômetro e calcular o tempo total
tempo_final = time.time()
tempo_total_segundos = tempo_final - tempo_inicial

# Formatando o tempo para horas, minutos e segundos
horas = int(tempo_total_segundos // 3600)
minutos = int((tempo_total_segundos % 3600) // 60)
segundos = int(tempo_total_segundos % 60)

print("\n--- Relatório Final de Execução ---")
print(f"Tempo de Execução Total: {horas}h {minutos}min {segundos}s")

# 2. Parar o monitor e gerar o gráfico de uso de recursos
# A função plot() já chama o stop() e processa os dados
monitor.plot()